In [4]:
from transformers import AutoTokenizer, AutoModel
import numpy as np

In [5]:
# Load a pre-trained transformer model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [6]:
# Function to compute embeddings
def compute_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embedding

# Sample FAQ data (questions and answers)
faq_data = {
    "What is machine learning?": "Machine learning is a field of AI that uses algorithms to learn patterns from data.",
    "How does artificial intelligence work?": "Artificial intelligence works by simulating human intelligence through algorithms and models.",
    "What is natural language processing?": "Natural Language Processing (NLP) involves the interaction between computers and human language.",
    "What is the use of Python in AI?": "Python is widely used in AI due to its simplicity and the availability of powerful libraries.",
    "How do neural networks learn?": "Neural networks learn by adjusting weights using backpropagation and gradient descent."
}

# Compute embeddings for FAQ questions
faq_questions = list(faq_data.keys())
faq_embeddings = np.vstack([compute_embedding(q) for q in faq_questions])

# Semantic search function
def faq_search(query, faq_questions, faq_embeddings, faq_data, top_k=1):
    query_embedding = compute_embedding(query)
    similarities = np.dot(faq_embeddings, query_embedding.T).flatten()
    top_indices = similarities.argsort()[::-1][:top_k]
    return [(faq_questions[i], faq_data[faq_questions[i]], similarities[i]) for i in top_indices]




In [7]:
# Example query
query = "Tell me about neural networks and how they work."

# Perform semantic search
results = faq_search(query, faq_questions, faq_embeddings, faq_data)

# Display the result
print(f"Query: {query}")
print("\nBest Match:")
for question, answer, score in results:
    print(f"Q: {question}")
    print(f"A: {answer}")
    print(f"Similarity Score: {score:.4f}")

Query: Tell me about neural networks and how they work.

Best Match:
Q: How do neural networks learn?
A: Neural networks learn by adjusting weights using backpropagation and gradient descent.
Similarity Score: 30.2085
